## Questions I want to answer: 
1. Which stock fluctuates more? 
1. What is the correlation between different stocks?
1. What was the daily return of the stock on average?
1. Which stock has less risk?
1. Which stock has more risk?

In [49]:
#import dataset from yahoo
from pandas_datareader.data import DataReader

from datetime import datetime
import pandas as pd
import numpy as np

In [50]:
help(DataReader)

Help on function DataReader in module pandas_datareader.data:

DataReader(name, data_source=None, start=None, end=None, retry_count=3, pause=0.1, session=None, api_key=None)
    Imports data from a number of online sources.
    
    Currently supports Google Finance, St. Louis FED (FRED),
    and Kenneth French's data library, among others.
    
    Parameters
    ----------
    name : str or list of strs
        the name of the dataset. Some data sources (IEX, fred) will
        accept a list of names.
    data_source: {str, None}
        the data source ("iex", "fred", "ff")
    start : string, int, date, datetime, Timestamp
        left boundary for range (defaults to 1/1/2010)
    end : string, int, date, datetime, Timestamp
        right boundary for range (defaults to today)
    retry_count : {int, 3}
        Number of times to retry query request.
    pause : {numeric, 0.001}
        Time, in seconds, to pause between consecutive queries of chunks. If
        single value given 

In [51]:
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']



In [85]:
end = datetime.now()  #end date
# year, month, day
start = datetime(year=end.year - 2, month=end.month, day=end.day)


#get me stock data of google for year 2019
df = DataReader('GOOG', 'yahoo',start=start, end=end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-10-05,1173.500000,1145.119995,1167.500000,1157.349976,1184300,1157.349976
2018-10-08,1168.000000,1127.364014,1150.109985,1148.969971,1932400,1148.969971
2018-10-09,1154.349976,1137.572021,1146.150024,1138.819946,1308700,1138.819946
2018-10-10,1132.170044,1081.130005,1131.079956,1081.219971,2675700,1081.219971
2018-10-11,1106.400024,1068.270020,1072.939941,1079.319946,2949000,1079.319946


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 503 entries, 2018-10-05 to 2020-10-05
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       503 non-null    float64
 1   Low        503 non-null    float64
 2   Open       503 non-null    float64
 3   Close      503 non-null    float64
 4   Volume     503 non-null    int64  
 5   Adj Close  503 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 27.5 KB


In [87]:
df.describe

<bound method NDFrame.describe of                    High          Low         Open        Close   Volume  \
Date                                                                      
2018-10-05  1173.500000  1145.119995  1167.500000  1157.349976  1184300   
2018-10-08  1168.000000  1127.364014  1150.109985  1148.969971  1932400   
2018-10-09  1154.349976  1137.572021  1146.150024  1138.819946  1308700   
2018-10-10  1132.170044  1081.130005  1131.079956  1081.219971  2675700   
2018-10-11  1106.400024  1068.270020  1072.939941  1079.319946  2949000   
...                 ...          ...          ...          ...      ...   
2020-09-29  1476.662964  1458.805054  1470.390015  1469.329956   978200   
2020-09-30  1489.750000  1459.880005  1466.800049  1469.599976  1700600   
2020-10-01  1499.040039  1479.209961  1484.270020  1490.089966  1779500   
2020-10-02  1483.199951  1450.920044  1462.030029  1458.420044  1282400   
2020-10-05  1488.209961  1464.270020  1466.209961  1486.020020  10

# feature engineering 

In [88]:
def generate_features(df):
    '''
    Generate features based on historical price and performance
    '''
    df = pd.DataFrame(df)
    #shift index by 1 to take values of previous days and assign to current values
    
    #compute shifted return
    df['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    #moving avg of daily returns over the past week
    df['moving_avg_5'] = df['return_1'].rolling(window=5).mean() #windows exclude weekends
    #moving avg of daily returns over the past month
    df['moving_avg_30'] = df['return_1'].rolling(window=21).mean()
    #moving avg of daily returns over the past year
    df['moving_avg_360'] = df['return_1'].rolling(window=252).mean()
    

    df = df.dropna(axis=0)
    
    print(len(df['moving_avg_360']))
    print(df['moving_avg_360'].count())
    return df


generate_features(df)

250
250


,High,Low,Open,Close,Volume,Adj Close,return_1,moving_avg_5,moving_avg_30,moving_avg_360
Date,,,,,,,,,,
2019-10-09,1208.349976,1197.630005,1199.349976,1202.310059,867700,1202.310059,-0.015360,-0.002547,-0.000545,0.000278
2019-10-10,1215.000000,1197.339966,1198.579956,1208.670044,846600,1208.670044,0.011084,0.004395,-0.000080,0.000350
2019-10-11,1228.390015,1213.739990,1222.209961,1215.449951,1272700,1215.449951,0.005290,0.003549,-0.000387,0.000406
2019-10-14,1226.329956,1211.760010,1212.339966,1217.140015,867500,1217.140015,0.005609,0.001106,-0.000670,0.000629
2019-10-15,1247.329956,1220.400024,1220.400024,1243.010010,1381700,1243.010010,0.001390,0.001603,-0.000809,0.000642
...,...,...,...,...,...,...,...,...,...,...
2020-09-29,1476.662964,1458.805054,1470.390015,1469.329956,978200,1469.329956,0.013537,0.004825,-0.004955,0.000974
2020-09-30,1489.750000,1459.880005,1466.800049,1469.599976,1700600,1469.599976,0.003284,0.000689,-0.005093,0.001006
2020-10-01,1499.040039,1479.209961,1484.270020,1490.089966,1779500,1490.089966,0.000184,0.007584,-0.004788,0.001052


## Linear Regression

## Decision Tree Regression